In [58]:
## 위경도 변환
import os
import pandas as pd
from pyproj import CRS, transformer

In [59]:
#proj 세팅
# CRS 및 Transformer 설정 # grs80 utm-k 좌표계 # 경위도좌표  ## 문의게시판 tech&tips  2번
grs_fr = CRS('EPSG:5179')
wgs_to = CRS('EPSG:4326')
transformer = Transformer.from_crs(grs_fr, wgs_to, always_xy=True)

In [ ]:
#파일리스트 가져오기
folder_path = './gpsdata'
file_list = os.listdir(folder_path)
file_list

In [61]:
# 좌표 변환 함수 정의
def convert_coordinates(row):
    lon1, lat1 = transformer.transform(row['x1'], row['y1'])
    lon2, lat2 = transformer.transform(row['x2'], row['y2'])
    return pd.Series([lon1, lat1, lon2, lat2])

In [62]:
#샘플 변환 테스트
df_sejong = pd.read_csv('./gpsdata/match_build_sejong.txt', encoding='cp949', sep='|', header=None, low_memory=False)
df_sejong.describe()

,0,2,4,6,7,8,9,13,15,16,17,18,22,23,24,25,26,28,31,32
count,5.627000e+04,0.0,5.627000e+04,56270.0,56270.000000,56270.000000,56270.000000,5.627000e+04,56270.000000,56270.000000,56270.000000,56270.000000,56270.000000,55860.000000,5.586000e+04,55866.000000,5.586600e+04,0.0,56270.000000,0.0
mean,3.611032e+09,NaN,3.611042e+11,0.0,145.967460,5.966661,30048.984592,3.611035e+09,1.770251,0.045619,0.056798,4.497814,0.834868,979239.156176,1.840920e+06,979239.881634,1.840919e+06,NaN,0.076506,NaN
std,7.200047e+03,NaN,6.374264e+05,0.0,344.807207,13.481075,32.594282,6.786042e+03,3.538893,0.295454,0.263891,7.146750,0.371303,4999.019240,8.074736e+03,4999.661648,8.072509e+03,NaN,0.265809,NaN
min,3.611010e+09,NaN,3.611010e+11,0.0,1.000000,0.000000,30000.000000,3.611025e+09,0.000000,0.000000,0.000000,1.000000,0.000000,967499.421769,1.823360e+06,967500.491468,1.823357e+06,NaN,0.000000,NaN
25%,3.611031e+09,NaN,3.611033e+11,0.0,22.000000,0.000000,30019.000000,3.611032e+09,1.000000,0.000000,0.000000,1.000000,1.000000,975323.439233,1.834754e+06,975316.437795,1.834755e+06,NaN,0.000000,NaN
50%,3.611034e+09,NaN,3.611046e+11,0.0,51.000000,0.000000,30049.000000,3.611035e+09,1.000000,0.000000,0.000000,3.000000,1.000000,979544.536338,1.840349e+06,979546.998701,1.840346e+06,NaN,0.000000,NaN
75%,3.611036e+09,NaN,3.611046e+11,0.0,125.000000,7.000000,30071.000000,3.611037e+09,1.000000,0.000000,0.000000,4.000000,1.000000,982001.031575,1.845484e+06,981994.327111,1.845471e+06,NaN,0.000000,NaN
max,3.611039e+09,NaN,3.611049e+11,0.0,13040.000000,259.000000,30154.000000,3.611058e+09,48.000000,7.000000,2.000000,85.000000,1.000000,991864.920765,1.859427e+06,991858.450474,1.859423e+06,NaN,1.000000,NaN


In [63]:
# 필요한 컬럼 선택
cols_to_convert = df_sejong.iloc[:, [23, 24, 25, 26]]
cols_to_convert.columns = ['x1', 'y1', 'x2', 'y2']  # 임시 컬럼명 지정

# 새 데이터프레임 생성
converted_df = cols_to_convert.apply(convert_coordinates, axis=1)
#converted_df.columns = ['경도x1', '위도y1', '경도x2', '위도y2']

# 변환된 데이터를 CSV 파일로 저장
converted_df.to_csv('converted_coordinates.csv', index=False)

print(f"좌표 변환 완료 및 CSV 파일 저장 완료")

좌표 변환 완료 및 CSV 파일 저장 완료


In [ ]:
#각 파일별로 불러와서 저장하기
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path, encoding='cp949', sep='|', header=None, low_memory=False)
    print(f'불러오기 완료: {file_name}')
    # 필요한 컬럼 선택
    cols_to_convert = df.iloc[:, [23, 24, 25, 26]]
    cols_to_convert.columns = ['x1', 'y1', 'x2', 'y2']  # 임시 컬럼명 지정
    print(f'컬럼선택 완료: {file_name}')
    
    # 변환하기
    converted_df = cols_to_convert.apply(convert_coordinates, axis=1)
    converted_df.columns = ['경도x1', '위도y1', '경도x2', '위도y2']
    
    # 변환된 데이터를 CSV 파일로 저장
    converted_df.to_csv(f'{file_name}_converted.csv', index=False)
    print(f'저장 완료: {file_name}')
